
# RAG over a Google Sheet TSV (with DeepSeek API) — Colab Notebook

This notebook lets you run **Retrieval-Augmented Generation (RAG)** over a **Google Sheet exported as TSV** (or any TSV URL) and use **DeepSeek** for the generation step.

**What you get:**
- No need for Google Sheets API — just a TSV link.
- Local, zero-cost embeddings (SentenceTransformers) + FAISS index.
- Controls for `k` (how many results you retrieve), chunking, max context size, temperature, etc.
- Clear, educational notes throughout so you can learn and tweak.

> **Tip:** In Google Sheets: File → Share → Publish to the web → Tab-separated values (TSV) link.



## 1) Setup

We install the minimal dependencies:
- `pandas` for loading TSV.
- `sentence-transformers` for local embeddings (free).
- `faiss-cpu` as a fast vector index.
- `ipywidgets` for interactive controls.
- `requests` to call DeepSeek's API.



In [ ]:

# If running locally or on Colab, uncomment as needed.
!pip -q install pandas sentence-transformers faiss-cpu ipywidgets requests
# Enable widgets in Colab
from google.colab import output
output.enable_custom_widget_manager()



## 2) DeepSeek API key & config

- We'll call the **DeepSeek Chat Completions** endpoint.
- Enter your API key below (it won't be printed).
- You can tweak model, temperature, and max tokens later.


In [ ]:

import getpass
from IPython.display import display
import ipywidgets as widgets

# Enter your DeepSeek API key here (recommended: use a secrets manager when possible)
deepseek_api_key = widgets.Password(
    description='API Key:',
    placeholder='sk-...',
    layout=widgets.Layout(width='50%')
)

# Advanced: base URL & model (defaults should work)
deepseek_base_url = widgets.Text(
    description='Base URL:',
    value='https://api.deepseek.com',
    layout=widgets.Layout(width='70%')
)

deepseek_model = widgets.Text(
    description='Model:',
    value='deepseek-chat',
    layout=widgets.Layout(width='50%')
)

display(deepseek_api_key, deepseek_base_url, deepseek_model)



## 3) Load your TSV

Paste a link to a **TSV**. This can be a Google Sheet published as TSV or any TSV hosted online.

**Notes:**
- No Google Sheets API needed.
- If your sheet contains headers in the first row, we will use them as column names.
- You can choose which columns to index (for retrieval) and which to display as sources.


In [ ]:

import pandas as pd
from IPython.display import display
import ipywidgets as widgets

tsv_url = widgets.Text(
    description='TSV URL:',
    placeholder='https://.../your.tsv',
    layout=widgets.Layout(width='90%')
)

load_button = widgets.Button(description='Load TSV', button_style='primary', icon='download')

out_load = widgets.Output()

def on_load_clicked(b):
    out_load.clear_output()
    with out_load:
        if not tsv_url.value.strip():
            print("Please provide a TSV URL.")
            return
        try:
            df = pd.read_csv(tsv_url.value.strip(), sep='\t', dtype=str, keep_default_na=False, on_bad_lines='skip')
            print(f"Loaded {len(df)} rows, {len(df.columns)} columns.")
            display(df.head(3))
            # store globally
            globals()['_rag_df'] = df
        except Exception as e:
            print("Failed to load TSV:", e)

load_button.on_click(on_load_clicked)

display(tsv_url, load_button, out_load)



## 4) Choose columns & chunking strategy

**What gets embedded?**  
The RAG index needs a **text representation** of each row. We typically join multiple columns into one chunk string.  

- **Columns to index**: We’ll concatenate these to form the text we embed and search over.  
- **Columns to show**: These will be presented with the answer as “sources.”  

**Chunking**: For spreadsheets, a good default is **one row = one chunk**. If your rows are very long, you can add a **max characters per chunk** to avoid huge context. We also include a **max context tokens** control to keep prompts small and cheap.


In [ ]:

import ipywidgets as widgets
from IPython.display import display

def get_df():
    df = globals().get('_rag_df', None)
    if df is None:
        raise RuntimeError("No TSV loaded yet. Go back to step 3.")
    return df

def make_multiselect(options, description):
    return widgets.SelectMultiple(
        options=options,
        description=description,
        layout=widgets.Layout(width='90%', height='150px')
    )

setup_button = widgets.Button(description='Configure Columns', icon='cog')
out_setup = widgets.Output()

index_cols_widget = None
show_cols_widget = None
id_col_widget = None

max_chars_per_chunk = widgets.IntSlider(description="Max chars/chunk", value=1000, min=200, max=5000, step=100)
max_context_tokens = widgets.IntSlider(description="Max context tokens", value=1200, min=200, max=6000, step=100)

def on_setup_clicked(b):
    global index_cols_widget, show_cols_widget, id_col_widget
    out_setup.clear_output()
    with out_setup:
        df = get_df()
        columns = list(df.columns)
        index_cols_widget = make_multiselect(columns, "Index cols")
        show_cols_widget = make_multiselect(columns, "Show cols")
        id_col_widget = widgets.Dropdown(options=['<row_number>'] + columns, description='ID column')
        display(index_cols_widget, show_cols_widget, id_col_widget, max_chars_per_chunk, max_context_tokens)

display(setup_button, out_setup)
setup_button.on_click(on_setup_clicked)



## 5) Build embeddings & FAISS index

We use `sentence-transformers/all-MiniLM-L6-v2` (free, fast, solid baseline).  
We **normalize vectors** and use **inner product (cosine)** in FAISS.

> Educational note: **k (top-k)** controls how many chunks to fetch before building the prompt. Too high → expensive & noisy. Too low → risk missing relevant context. Start with **k=3–5**.


In [ ]:

import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from IPython.display import display
import ipywidgets as widgets

build_button = widgets.Button(description='Build Index', button_style='success', icon='hammer')
out_build = widgets.Output()

# Controls for retrieval
k_slider = widgets.IntSlider(description='k (top results)', value=4, min=1, max=15, step=1)
min_sim_threshold = widgets.FloatSlider(description='Min similarity', value=0.2, min=0.0, max=0.99, step=0.01)

def row_to_text(row, cols):
    # Join selected columns for the chunk text
    parts = []
    for c in cols:
        parts.append(f"{c}: {row.get(c, '')}")
    return " | ".join(parts)

def trim_text(s, max_chars):
    return s if len(s) <= max_chars else s[:max_chars] + " …"

def on_build_clicked(b):
    out_build.clear_output()
    with out_build:
        try:
            df = get_df()
        except RuntimeError as e:
            print(e)
            return

        if k_slider.value < 1:
            print("k must be >= 1")
            return

        if globals().get('index_cols_widget', None) is None or len(index_cols_widget.value) == 0:
            print("Please choose at least one column to index (Step 4).")
            return

        idx_cols = list(index_cols_widget.value)
        show_cols = list(show_cols_widget.value) if show_cols_widget and len(show_cols_widget.value)>0 else idx_cols
        use_id_col = id_col_widget.value if id_col_widget else '<row_number>'

        # Build documents (one row = one chunk)
        docs = []
        meta = []  # store row id + raw fields to show later
        for i, row in get_df().iterrows():
            row_dict = row.to_dict()
            chunk_text = trim_text(row_to_text(row_dict, idx_cols), max_chars_per_chunk.value)
            row_id = row_dict.get(use_id_col, i) if use_id_col != '<row_number>' else i
            docs.append(chunk_text)
            meta.append({
                'row_index': i,
                'row_id': row_id,
                'show': {c: row_dict.get(c, '') for c in show_cols}
            })

        print(f"Building embeddings for {len(docs)} chunks...")
        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        vecs = model.encode(docs, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)

        dim = vecs.shape[1]
        index = faiss.IndexFlatIP(dim)  # inner product; with normalized vectors, equals cosine sim
        index.add(vecs)

        globals()['_rag_index'] = index
        globals()['_rag_vecs'] = vecs
        globals()['_rag_docs'] = docs
        globals()['_rag_meta'] = meta
        globals()['_rag_model'] = model
        globals()['_rag_setup'] = dict(idx_cols=idx_cols, show_cols=show_cols, use_id_col=use_id_col)

        print("Index built.")
        print("— Dimensions:", dim)
        print("— Chunks:", len(docs))
        print("— Example chunk:", docs[0][:200])

display(k_slider, min_sim_threshold, build_button, out_build)
build_button.on_click(on_build_clicked)



## 6) Ask questions (RAG) and generate with DeepSeek

**How it works:**
1. We embed your query locally.
2. We retrieve top-`k` results from FAISS (filtering by min similarity if set).
3. We **truncate the concatenated context** to keep within your **max context tokens** (approximate).
4. We build a prompt that tells the model to answer **using only the provided context**.
5. We call **DeepSeek** for the final answer.

You can tweak:
- `k (top results)`
- `min similarity`
- `max context tokens`
- `temperature` and `max tokens` for generation


In [ ]:

import math
import requests
from IPython.display import display, Markdown
import ipywidgets as widgets

query_box = widgets.Textarea(
    description='Question:',
    placeholder='Ask a question about your TSV data...',
    layout=widgets.Layout(width='90%', height='80px')
)

temperature = widgets.FloatSlider(description='Temperature', value=0.2, min=0.0, max=1.5, step=0.05)
gen_max_tokens = widgets.IntSlider(description='Gen max tokens', value=512, min=32, max=4096, step=32)

run_button = widgets.Button(description='Run RAG', button_style='primary', icon='play')
out_run = widgets.Output()

def approx_token_len(text):
    # crude approx: 1 token ~ 4 chars (varies by tokenizer/model)
    return max(1, math.ceil(len(text) / 4))

def build_context(query_vec, k, min_sim):
    index = globals().get('_rag_index', None)
    vecs = globals().get('_rag_vecs', None)
    docs = globals().get('_rag_docs', None)
    meta = globals().get('_rag_meta', None)
    model = globals().get('_rag_model', None)

    if index is None:
        raise RuntimeError("No index built yet. Run Step 5.")
    if model is None:
        raise RuntimeError("Embedding model not found. Rebuild index.")

    # encode query
    qv = model.encode([query_box.value], convert_to_numpy=True, normalize_embeddings=True)
    # search
    D, I = index.search(qv, min(k, index.ntotal))
    I = I[0]; D = D[0]

    # filter by similarity threshold
    pairs = [(int(i), float(d)) for i, d in zip(I, D) if float(d) >= min_sim]
    if not pairs:
        return "", []

    # take top-k after filtering
    top = pairs[:k]
    # format context blocks with provenance
    blocks = []
    sources = []
    for idx, sim in top:
        md = meta[idx]
        show_kv = " | ".join(f"{k}: {v}" for k, v in md['show'].items())
        blocks.append(f"[row_id={md['row_id']}; sim={sim:.3f}] {docs[idx]}")
        sources.append({'row_id': md['row_id'], 'similarity': sim, **md['show']})

    context = "\n\n".join(blocks)
    return context, sources

def truncate_context_by_tokens(context, max_tokens):
    # very rough truncation based on char length; keeps head
    approx = approx_token_len(context)
    if approx <= max_tokens:
        return context
    # truncate head
    target_chars = max_tokens * 4
    return context[:target_chars] + "\n\n… (truncated)"

def call_deepseek_chat(system_prompt, user_prompt):
    base = deepseek_base_url.value.rstrip('/')
    url = f"{base}/chat/completions" if not base.endswith("/v1") else f"{base}/chat/completions"
    # If your endpoint requires /v1/chat/completions, set base to https://api.deepseek.com/v1
    headers = {
        "Authorization": f"Bearer {deepseek_api_key.value.strip()}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": deepseek_model.value.strip() or "deepseek-chat",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": float(temperature.value),
        "max_tokens": int(gen_max_tokens.value),
        "stream": False
    }
    resp = requests.post(url, headers=headers, json=payload, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    # OpenAI-compatible shape
    return data["choices"][0]["message"]["content"]

def format_sources_table(sources):
    if not sources:
        return "No sources."
    # simple Markdown table
    headers = list(sources[0].keys())
    lines = ["|" + "|".join(headers) + "|", "|" + "|".join(["---"]*len(headers)) + "|"]
    for s in sources:
        row = [str(s.get(h, "")) for h in headers]
        lines.append("|" + "|".join(row) + "|")
    return "\n".join(lines)

def on_run_clicked(b):
    out_run.clear_output()
    with out_run:
        if not deepseek_api_key.value.strip():
            print("Please enter your DeepSeek API key in Step 2.")
            return
        if not query_box.value.strip():
            print("Please enter a question.")
            return
        try:
            ctx, sources = build_context(
                query_vec=None,
                k=int(k_slider.value),
                min_sim=float(min_sim_threshold.value)
            )
        except Exception as e:
            print("Error during retrieval:", e)
            return

        if not ctx.strip():
            print("No matching context found above your similarity threshold. Try lowering the threshold or increasing k.")
            return

        ctx_trunc = truncate_context_by_tokens(ctx, int(max_context_tokens.value))

        system_prompt = (
            "You are a careful assistant. Answer the user's question using ONLY the provided context. "
            "If the answer isn't contained in the context, say you don't know."
        )
        user_prompt = f"Context:\n{ctx_trunc}\n\nQuestion:\n{query_box.value}\n\nAnswer concisely:"

        try:
            answer = call_deepseek_chat(system_prompt, user_prompt)
        except requests.HTTPError as e:
            print("DeepSeek API error:", e.response.text[:500])
            return
        except Exception as e:
            print("Failed calling DeepSeek API:", e)
            return

        display(Markdown("### Answer"))
        display(Markdown(answer))

        display(Markdown("### Sources (top-k by similarity)"))
        display(Markdown(format_sources_table(sources)))

display(query_box, temperature, gen_max_tokens, run_button, out_run)
run_button.on_click(on_run_clicked)



## 7) Educational notes — the important knobs

- **k (top results):** How many chunks you retrieve. Start at **3–5**. Higher k increases cost and can add noise; lower k risks missing relevant info.  
- **Min similarity:** Filter out low-quality matches. Raise it (e.g., 0.3–0.5) to be stricter; lower it if you’re missing answers.  
- **Max chars per chunk:** Avoids creating huge chunks from very long rows. Helps keep prompts small.  
- **Max context tokens:** Hard cap for how much context you pass to the model; keeps cost/latency predictable.  
- **Temperature:** Lower is more deterministic; higher can be more creative but less faithful to context.  
- **Prompting style:** We instruct the model to answer **only** from context. This reduces hallucinations.

**When to consider re-ranking:**  
If your dataset is big or noisy, consider a two-stage approach: retrieve k=10, then re-rank down to 3 using a local cross-encoder (e.g., `cross-encoder/ms-marco-MiniLM-L-6-v2`). This adds compute but can boost precision.

**Scaling tips:**  
- Cache embeddings and the FAISS index to disk.  
- Only re-embed changed rows by hashing row contents.  
- For *very* large sheets, move to a persistent vector store (e.g., Qdrant) but keep the same interface.



## 8) (Optional) Save / Load the index

Use these helpers to persist your work between sessions. You’ll still need to re-load the TSV to reconstruct metadata.


In [ ]:

import pickle, os, faiss

save_prefix = 'rag_index'

def save_index(prefix=save_prefix):
    index = globals().get('_rag_index', None)
    vecs = globals().get('_rag_vecs', None)
    docs = globals().get('_rag_docs', None)
    meta = globals().get('_rag_meta', None)
    setup = globals().get('_rag_setup', None)
    if index is None:
        print("No index to save.")
        return
    faiss.write_index(index, f"{prefix}.faiss")
    with open(f"{prefix}.pkl", "wb") as f:
        pickle.dump(dict(vecs=vecs, docs=docs, meta=meta, setup=setup), f)
    print(f"Saved: {prefix}.faiss & {prefix}.pkl")

def load_index(prefix=save_prefix):
    if not os.path.exists(f"{prefix}.faiss") or not os.path.exists(f"{prefix}.pkl"):
        print("Saved files not found.")
        return
    index = faiss.read_index(f"{prefix}.faiss")
    with open(f"{prefix}.pkl", "rb") as f:
        data = pickle.load(f)
    globals()['_rag_index'] = index
    globals()['_rag_vecs'] = data['vecs']
    globals()['_rag_docs'] = data['docs']
    globals()['_rag_meta'] = data['meta']
    globals()['_rag_setup'] = data['setup']
    print("Index loaded.")

# Buttons
save_btn = widgets.Button(description='Save Index', icon='save')
load_btn = widgets.Button(description='Load Index', icon='folder-open')
out_persist = widgets.Output()

def on_save(b):
    out_persist.clear_output()
    with out_persist:
        save_index()

def on_load(b):
    out_persist.clear_output()
    with out_persist:
        load_index()

save_btn.on_click(on_save)
load_btn.on_click(on_load)

display(widgets.HBox([save_btn, load_btn]), out_persist)
